<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/ai_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="blue">**Training and Tuning**

100+ Fine-tuning Unsloth AI notebooks: https://github.com/unslothai/notebooks/

##### *Create a LoRA adapter*

Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint

https://huggingface.co/meta-llama/Llama-3.1-8B

https://huggingface.co/models?other=base_model:quantized:meta-llama/Llama-3.1-8B

https://huggingface.co/unsloth/Meta-Llama-3.1-8B-bnb-4bit

Paper: https://arxiv.org/abs/2106.09685

Video: https://www.youtube.com/watch?v=Us5ZFp16PaU&t=393s

**YT PEFT Finetune-Bloom7B-tagger**

Original: https://colab.research.google.com/drive/14xo6sj4dARk8lXZbOifHEn1f_70qNAwy?usp=sharing#scrollTo=4iwHGzKBN6wk


In [ ]:
# Setup the model

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

In [ ]:
# Freezing the original weights

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
# Setting up the LoRa Adapters

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
# Data

import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")

In [ ]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

In [ ]:
data['train'][0]

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [ ]:
# Training

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
# Share adapters on the 🤗 Hub

model.push_to_hub("samwit/bloom-7b1-lora-tagger",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

In [ ]:
# Load adapters from the Hub

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "samwit/bloom-7b1-lora-tagger"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

**Llama-3.1 8b + Unsloth 2x faster finetuning**

https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=QmUBVEnvCDJv


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
# Now add LoRA adapters so we only need to update 1 to 10% of all parameters!
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

##### *Appendix*

Finetuning LLM
Official documentation on tuning large language models: https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-models
Codelab tutorial on fine-tuning LLMs with Vertex AI: https://codelabs.developers.google.com/llm-finetuning-supervised
Medium article providing insights into fine-tuning with Vertex AI: https://medium.com/google-cloud/fine-tuning-large-language-models-how-vertex-ai-takes-llms-to-the-next-level-3c113f4007da

https://colab.sandbox.google.com/drive/1CHtIWodkATMxWSkSdaZj60EkFTXymq26?resourcekey=0-YVhm1YZ9excXMSpaTTtxhA#scrollTo=35615fa2-c923-4bfd-b194-5265bbe81ce1

managed tuning OSS



Model Garden

PEFT: https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models

Example: Mistral: https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mistral_peft_tuning.ipynb

Example: LLama3 https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama3_finetuning.ipynb

Managed PEFT for open-weight models: pending

Full finetuning: released

DPO - Direct Preference Optimization: directly optimizes the LLM's policy using a dataset of human preferences. https://arxiv.org/pdf/2305.18290.pdf (pending)

Continuous Pretraining: bring your own custom LoRA (end Q2) and tokenspace


https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-models

**Open source:** Ray, Horovod (training), Axolotl (finetune multiple GPU), Unsloth (finetune single GPU), Dask, Spark


Model Garden Tuning: Explore Models https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models


Vertex Training: Pricing for training (https://cloud.google.com/vertex-ai/pricing#custom-trained_models) for example SVT for Gemini (https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-supervised-tuning#supported_models)


Ray on Vertex (managed https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview) or Ray on GKE (with KubeRay https://cloud.google.com/kubernetes-engine/docs/add-on/ray-on-gke/concepts/overview), example see Gemma on Ray (https://developers.googleblog.com/en/get-started-with-gemma-on-ray-on-vertex-ai/), Scale AI on Ray on Vertex AI  (https://medium.com/google-cloud/ray-on-vertex-ai-lets-get-it-started-7a9f8360ea25)
